In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

In [2]:
embeddings = np.load("embeddings_sample.npy")
vocab = pickle.load(open("data/vocab_full.pkl",'rb'))

t_pos = pd.read_table("data/train_pos_full.txt", header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_pos['label'] = 1
t_neg = pd.read_table("data/train_neg_full.txt", header=None, names=['tweet'], dtype=str,on_bad_lines='skip')
t_neg['label'] = -1
df = pd.concat((t_pos,t_neg))
df.shape, embeddings.shape, len(vocab)

((2458295, 2), (101298, 20), 101298)

In [3]:
max_len = max([len(tweet.split(' ')) for tweet in df.tweet])
print(f'{max_len=}')

max_len=3114


In [8]:
tokenized_tweets_np = np.zeros((len(df),20))
for i, tweet in enumerate(tqdm(df.tweet)):
    tweet_len = len(tweet.split(' '))
    tweet_embedding=np.zeros((tweet_len,20))
    for j, word in enumerate(tweet.split(' ')):
        if word not in vocab.keys():
            continue
        tweet_embedding[j,:] = embeddings[vocab.get(word),:]
    
    tokenized_tweets_np[i,:] = tweet_embedding.mean(0)
tokenized_tweets_np.shape

100%|██████████| 2458295/2458295 [01:22<00:00, 29911.43it/s]


(2458295, 20)

In [9]:
X = tokenized_tweets_np
y = df.label.values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.20, random_state = 42)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((1966636, 20), (491659, 20), (1966636,), (491659,))

In [12]:
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)

LogisticRegression(max_iter=500)

In [13]:
print(f'logreg training accuracy = {logreg.score(X_train, y_train):.4f}')
print(f'logreg val accuracy = {logreg.score(X_val, y_val):.4f}')

logreg training accuracy = 0.5675
logreg val accuracy = 0.5662


In [14]:
SVC = LinearSVC(verbose=1, max_iter=2000)
SVC.fit(X_train,  y_train)

[LibLinear].............................................................*.
optimization finished, #iter = 620
Objective value = -1904587.067851
nSV = 1966540


LinearSVC(max_iter=2000, verbose=1)

In [15]:
print(f'SVM training accuracy = {SVC.score(X_train, y_train):.4f}')
print(f'SVM val accuracy = {SVC.score(X_val, y_val):.4f}')

SVM training accuracy = 0.5677
SVM val accuracy = 0.5664
